In [10]:
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
from IPython.core.display import display
from numpy.core.defchararray import isdigit
from sklearn.metrics import accuracy_score

from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

from multi_imbalance.datasets import load_datasets
from multi_imbalance.resampling.SOUP import SOUP
from multi_imbalance.resampling.MDO import MDO
from multi_imbalance.resampling.GlobalCS import GlobalCS

from imblearn.metrics import geometric_mean_score
from imblearn.over_sampling import SMOTE
from multi_imbalance.resampling.spider import SPIDER3

from sklearn.neighbors import KNeighborsClassifier
maj_int_min = {
    'balance_scale' : {
        'maj': [2, 1],
        'int': [],
        'min': [0]
    }, 
    'cleveland': {
        'maj': [0],
        'int': [1],
        'min': [2,3,4]
    }, 
    'cmc': {
        'maj': [0],
        'int': [2],
        'min': [1]
    }, 
    'dermatology': {
        'maj': [0],
        'int': [2,1,4,3],
        'min': [5]
    }, 
    'ecoli': {
        'maj': [0,1],
        'int': [7,4,5],
        'min': [6,3,2]
    }, 
    'glass': {
        'maj': [1,0],
        'int': [5],
        'min': [2,3,4]
    }, 
    'hayes_roth': {
        'maj': [0,1],
        'int': [],
        'min': [2]
    }, 
    'new_thyroid': {
        'maj': [0],
        'int': [],
        'min': [1,2]
    }, 
    'winequailty_red': {
        'maj': [2,3],
        'int': [4],
        'min': [1,5,0]
    }, 
    'yeast': {
        'maj': [0,7],
        'int': [6, 5],
        'min': [4,3,2,9,8,1]
    }
}
from IPython.display import clear_output
clear_output(wait=True)

In [2]:
def resample_data(resample, seed, X_train, y_train, no_classes, dataset_name):
    if resample == 'base':
        X_train_resampled, y_train_resampled = X_train, y_train
    elif resample=='soup':
        soup = SOUP()
        X_train_resampled, y_train_resampled = soup.fit_transform(np.copy(X_train), np.copy(y_train))
    elif resample=='global':
        global_cs = GlobalCS()
        X_train_resampled, y_train_resampled = global_cs.fit_transform(np.copy(X_train), np.copy(y_train), shuffle=False)
    elif resample=='smote':
        smote = SMOTE(random_state=seed)
        X_train_resampled, y_train_resampled = smote.fit_sample(np.copy(X_train), np.copy(y_train))
    elif resample=='mdo':
        mdo = MDO(k=9, k1_frac=0.1, seed=seed)
        X_train_resampled, y_train_resampled = mdo.fit_transform(np.copy(X_train), np.copy(y_train))
    elif resample=='spider':
        cost = np.ones((no_classes, no_classes))
        np.fill_diagonal(cost, 0)
        clf = SPIDER3(k=5, cost=cost, majority_classes=maj_int_min[dataset_name]['maj'], intermediate_classes=maj_int_min[dataset_name]['int'], minority_classes=maj_int_min[dataset_name]['min'])
        X_train_resampled, y_train_resampled = clf.fit_transform(X_train.astype(np.float64), y_train)
    return X_train_resampled, y_train_resampled


def test_resampling(classifier, res, dataset_values, dataset_name):
    X, y = dataset_values.data, dataset_values.target
    no_classes = np.unique(y).size
    result_data = defaultdict(int)
    run_data = defaultdict(lambda: defaultdict(list)) # {metric: {run_number: [scores]}}
    for i in range(10):
        skf = StratifiedKFold(n_splits=5, shuffle=True,random_state=i)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            X_train_resampled, y_train_resampled = resample_data(res, i, X_train, y_train, no_classes, dataset_name)
            
            if classifier == 'knn':
                clf = KNeighborsClassifier(n_neighbors=5)
            elif classifier == 'tree':
                clf = DecisionTreeClassifier(random_state=i)
            
            clf.fit(X_train_resampled, y_train_resampled)
            y_pred = clf.predict(X_test)
            gmean = geometric_mean_score(y_test, y_pred, correction=0.001)
            run_data['g_mean'][str(i)].append(gmean)
    
    def get_score_from_metric(run_data, metric):
        runs = run_data[metric]
        runs_scores_list = list(runs.values()) #[[one run k-foledscores],[..]]
        result = np.mean(list(map(np.mean, runs_scores_list)))
        return result
            
    result_data['g_mean'] = get_score_from_metric(run_data, 'g_mean')
    return result_data


In [3]:
def provide_test_and_get_scores(datasets, clf):
    scores = defaultdict(dict)
    for dataset_name, dataset_values in datasets.items():
        clf_res_names =['base','global','smote','soup','mdo']
        # print(dataset_name)
        for resample in clf_res_names:
            result_data = test_resampling(clf, resample, dataset_values, dataset_name)
            scores[dataset_name][resample] = round(result_data['g_mean'],3)
    return scores


In [4]:
def print_scores(scores, only_read_dt = False):
    display("G-MEAN")
    df = pd.DataFrame(scores).T
    if only_read_dt:
        df = df.iloc[4:]
    display(df)
    
    # df.fillna(df.median(), inplace=True)
    display(pd.DataFrame(df.mean().sort_values(ascending=False),columns=['Mean G-mean']))
    display(pd.DataFrame(df.rank(axis=1,ascending=False).mean().sort_values(),columns=['Mean rank']))

In [5]:
datasets = load_datasets()


Testy dla drzewa,
Wszystkie zbiory danych:

In [6]:
score = provide_test_and_get_scores(datasets, 'tree')
print_scores(score)


'G-MEAN'

,base,global,smote,soup,mdo
1czysty-cut,0.939,0.946,0.955,0.957,0.965
2delikatne-cut,0.698,0.699,0.744,0.795,0.772
3mocniej-cut,0.492,0.482,0.496,0.578,0.585
4delikatne-bezover-cut,0.771,0.768,0.815,0.894,0.830
balance-scale,0.154,0.123,0.168,0.621,0.162
cleveland,0.127,0.096,0.142,0.139,0.098
cleveland_v2,0.113,0.110,0.129,0.162,0.090
cmc,0.440,0.451,0.444,0.466,0.439
dermatology,0.925,0.940,0.946,0.933,0.948
glass,0.463,0.486,0.554,0.606,0.598


,Mean G-mean
soup,0.646353
mdo,0.613706
smote,0.606824
base,0.582353
global,0.579412


,Mean rank
smote,2.294118
soup,2.352941
mdo,2.411765
global,3.941176
base,4.000000


Drzewo, tylko rzeczywiste zbiory danych:

In [7]:
print_scores(score,only_read_dt=True)


'G-MEAN'

,base,global,smote,soup,mdo
balance-scale,0.154,0.123,0.168,0.621,0.162
cleveland,0.127,0.096,0.142,0.139,0.098
cleveland_v2,0.113,0.110,0.129,0.162,0.090
cmc,0.440,0.451,0.444,0.466,0.439
dermatology,0.925,0.940,0.946,0.933,0.948
glass,0.463,0.486,0.554,0.606,0.598
hayes-roth,0.837,0.843,0.841,0.835,0.842
new_ecoli,0.708,0.707,0.723,0.714,0.758
new_led7digit,0.754,0.757,0.762,0.760,0.753
new_vehicle,0.900,0.894,0.890,0.886,0.899


,Mean G-mean
soup,0.597231
smote,0.562000
mdo,0.560077
base,0.538462
global,0.535000


,Mean rank
smote,2.076923
soup,2.615385
mdo,2.692308
global,3.769231
base,3.846154


Testy dla knn,
Wszystkie zbiory danych:

In [8]:
score = provide_test_and_get_scores(datasets, 'knn')
print_scores(score)


'G-MEAN'

,base,global,smote,soup,mdo
1czysty-cut,0.971,0.975,0.978,0.947,0.977
2delikatne-cut,0.704,0.760,0.761,0.789,0.801
3mocniej-cut,0.466,0.523,0.498,0.556,0.599
4delikatne-bezover-cut,0.812,0.852,0.861,0.889,0.875
balance-scale,0.193,0.267,0.420,0.678,0.684
cleveland,0.020,0.134,0.129,0.107,0.066
cleveland_v2,0.009,0.183,0.233,0.191,0.056
cmc,0.482,0.476,0.481,0.510,0.479
dermatology,0.843,0.849,0.849,0.815,0.854
glass,0.201,0.625,0.621,0.609,0.499


,Mean G-mean
soup,0.633588
smote,0.621118
mdo,0.608765
global,0.590176
base,0.521412


,Mean rank
smote,2.382353
mdo,2.558824
soup,2.794118
global,2.911765
base,4.352941


knn (k=5), tylko rzeczywiste zbiory danych:

In [9]:
print_scores(score,only_read_dt=True)


'G-MEAN'

,base,global,smote,soup,mdo
balance-scale,0.193,0.267,0.420,0.678,0.684
cleveland,0.020,0.134,0.129,0.107,0.066
cleveland_v2,0.009,0.183,0.233,0.191,0.056
cmc,0.482,0.476,0.481,0.510,0.479
dermatology,0.843,0.849,0.849,0.815,0.854
glass,0.201,0.625,0.621,0.609,0.499
hayes-roth,0.559,0.614,0.627,0.611,0.611
new_ecoli,0.814,0.775,0.807,0.817,0.824
new_led7digit,0.757,0.441,0.727,0.746,0.774
new_vehicle,0.849,0.863,0.859,0.821,0.852


,Mean G-mean
soup,0.583846
smote,0.573923
mdo,0.545923
global,0.532538
base,0.454692


,Mean rank
smote,2.269231
global,2.730769
soup,2.884615
mdo,2.884615
base,4.230769
